# Kaggle - House Price Prediction

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics
import datetime

In [3]:
PATH = "data/houseprice/"

In [4]:
os.listdir(f'{PATH}')

['data_description.txt',
 'sample_submission.csv',
 'sample_submission.csv.gz',
 'test.csv',
 'test.csv.gz',
 'train.csv',
 'train.csv.gz']

In [5]:
train = pd.read_csv(f'{PATH}train.csv')
test = pd.read_csv(f'{PATH}test.csv')

In [6]:
train = train.drop(['Id'],axis=1)
X_test = test.drop(['Id'],axis=1)

In [7]:
X_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [8]:
train.SalePrice = np.log(train.SalePrice)

In [9]:
train_cats(train)
train_cats(X_test)

In [10]:
train.shape

(1460, 80)

In [11]:
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,12.247694
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,12.109011
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,12.317167
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,11.849398
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,12.429216


In [12]:
x, y, nas = proc_df(train, 'SalePrice')
X_test, _, _ = proc_df(X_test)

In [13]:
x.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_na,MasVnrArea_na,GarageYrBlt_na
0,60,4,65.0,8450,2,0,4,4,1,5,...,0,0,0,2,2008,9,5,False,False,False
1,20,4,80.0,9600,2,0,4,4,1,3,...,0,0,0,5,2007,9,5,False,False,False
2,60,4,68.0,11250,2,0,1,4,1,5,...,0,0,0,9,2008,9,5,False,False,False
3,70,4,60.0,9550,2,0,1,4,1,1,...,0,0,0,2,2006,9,1,False,False,False
4,60,4,84.0,14260,2,0,1,4,1,3,...,0,0,0,12,2008,9,5,False,False,False


In [14]:
X_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,MasVnrArea_na,BsmtFinSF1_na,BsmtFinSF2_na,BsmtUnfSF_na,TotalBsmtSF_na,BsmtFullBath_na,BsmtHalfBath_na,GarageYrBlt_na,GarageCars_na,GarageArea_na
0,20,3,80.0,11622,2,0,4,4,1,5,...,False,False,False,False,False,False,False,False,False,False
1,20,4,81.0,14267,2,0,1,4,1,1,...,False,False,False,False,False,False,False,False,False,False
2,60,4,74.0,13830,2,0,1,4,1,5,...,False,False,False,False,False,False,False,False,False,False
3,60,4,78.0,9978,2,0,1,4,1,5,...,False,False,False,False,False,False,False,False,False,False
4,120,4,43.0,5005,2,0,1,2,1,5,...,False,False,False,False,False,False,False,False,False,False


In [19]:
cols = [c for c in X_test.columns if c not in x.columns]

In [21]:
X_test = X_test.drop(cols,axis=1)

In [15]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 250  # same as Kaggle's test set size
n_trn = len(x)-n_valid
raw_train, raw_valid = split_vals(train, n_trn)
X_train, X_valid = split_vals(x, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((1210, 82), (1210,), (250, 82))

In [16]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    #result
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [27]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
%time m.fit(x, y)
print_score(m)

Wall time: 143 ms
[0.0789284635580323, 0.08289518849593165, 0.9616242998229456, 0.9527645495161442, 0.8630821351945025]


In [28]:
predicted = np.exp(m.predict(X_test))

In [29]:
print(predicted)

[123246.17239 156370.74888 181648.99273 ... 148658.79199 111915.49116 234751.61884]


In [30]:
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice': predicted})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)